In [1]:
import pandas as pd

In [2]:
weather_train = pd.read_csv("../data/raw/weather_train.csv")

In [4]:
weather_train.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [40]:
from meteocalc import feels_like, Temp, dew_point, wind_chill, heat_index

In [7]:
weather_train.shape

(139773, 9)

In [19]:
temp_format = weather_train.air_temperature.apply(lambda x : Temp(x, 'c'))

In [27]:
weather_train.loc[1,]["air_temperature"]

24.4

In [24]:
import math
def compute_humidity(row):
    CONSTANTS = dict(
        positive=dict(b=17.368, c=238.88),
        negative=dict(b=17.966, c=247.15),
    )
    T = row["air_temperature"]
    const = CONSTANTS['positive'] if T > 0 else CONSTANTS['negative']
    dp = row["dew_temperature"]
    pa = math.exp(dp * const['b'] / (const['c'] + dp))
    rel_humidity = pa * 100. * 1 / math.exp(const['b'] * T / (const['c'] + T))
    return(rel_humidity)
    

In [28]:
weather_train["relative_humidity"] = weather_train.apply(lambda x: compute_humidity(x), axis = 1)

In [29]:
weather_train.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,relative_humidity
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,73.810815
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5,81.870974
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,90.151953
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,96.971724
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,100.000000


In [33]:
max(weather_train.relative_humidity)

100.00000000000001

In [45]:
dew_point(temperature = weather_train["air_temperature"][1], humidity = weather_train["relative_humidity"][1]).f

69.97999999999999

In [47]:
weather_train["air_temp_f"] = weather_train["air_temperature"] * 9 / 5. + 32

In [48]:
def feels_like_custom(row):
    temperature = row["air_temp_f"]
    wind_speed = row["wind_speed"]
    humidity = row["relative_humidity"]
    fl = feels_like(temperature, wind_speed, humidity)
    out = fl.c
    return(out)
    

In [50]:
weather_train["feels_like_temp"] = weather_train.apply(lambda x : feels_like_custom(x), axis = 1)